In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

# Dados de Jogadores

### Acessar página e encontrar tabela

In [4]:
pagina_jogadores = urlopen("https://fbref.com/pt/equipes/83f55dbe/Vasco-da-Gama-Estatisticas")
bs_pagina_jogadores = BeautifulSoup(pagina_jogadores, 'html.parser')
tabela_jogadores = bs_pagina_jogadores.find("div", {'id':'div_stats_standard_10987'}).table.tbody

### Coletar dados e exportar CSV

In [6]:
dados_jogadores = [
    (
        linha.th["data-append-csv"],
        linha.find_all("td")[0].a.span.text[-3:],
        linha.find_all("td")[1].text
    )
    for linha in tabela_jogadores.find_all("tr")
]
# criar data frame e exportar csv
jogadores_df = pd.DataFrame(dados_jogadores,columns=['jogador','pais','posicao'])
jogadores_df.to_csv("jogadores.csv", index=False)

# Dados de Partidas

### Acessar página e encontrar tabela

In [7]:
pagina_partidas = urlopen("https://fbref.com/pt/equipes/83f55dbe/2021/partidas/s10987/schedule/Vasco-da-Gama-Resultados-e-Calendarios-Serie-B")
bs_pagina_partidas = BeautifulSoup(pagina_partidas, 'html.parser')
tabela_partidas = bs_pagina_partidas.find("div", {'id':'div_matchlogs_for'}).table.tbody

### Coletar dados e exportar CSV

In [8]:
dados_partidas = [
    (
        # data
        linha.th.a.text,
        # hora
        linha.find_all("td")[0].text,
        # rodada
        linha.find_all("td")[1].text[-2:],
        # mando
        linha.find_all("td")[3].text,
        # resultado
        linha.find_all("td")[4].text,
        # gols pro
        linha.find_all("td")[5].text,
        # gols contra
        linha.find_all("td")[6].text,
        # adversário
        linha.find_all("td")[7].a.text,
        # posse de bola
        linha.find_all("td")[8].text,
        # publico
        linha.find_all("td")[9].text,
        # capitao
        linha.find_all("td")[10].a.text,
        # arbitro
        linha.find_all("td")[12].text
    )
    for linha in tabela_partidas.find_all("tr")[:31]
]
# criar data frame e exportar csv
partidas_df = pd.DataFrame(dados_partidas,
                           columns=[
                               'data',
                               'hora',
                               'rodada',
                               'mando',
                               'resultado',
                               'gols pro',
                               'gols contra',
                               'adversário',
                               'posse de bola',
                               'publico',
                               'capitao',
                               'arbitro'
                           ])
partidas_df.to_csv("partidas.csv", index=False)

# Dados de Desempenho

### Encontrar URL da partida

In [9]:
url_partidas = [
    (
        # Rodada
        partida.find_all("td")[1].text[-2:],
        # URL
        partida.find_all("td")[13].a['href']
    ) 
    for partida in tabela_partidas.find_all("tr")[:31] # 31 rodadas ate o momento
]

### Acessar páginas de desempenho e obter dados de jogador

In [10]:
for rodada in url_partidas:
    numero_rodada = rodada[0]
    url = rodada[1]
    # obter página
    pagina_desempenho = urlopen(f'https://fbref.com{url}')
    # simular tempo de execução de usuario comum
    sleep(60)
    bs_pagina_desempenho = BeautifulSoup(pagina_desempenho, 'html.parser')
    # localizar tabela de desempenho do Vasco
    titulo_tabela = bs_pagina_desempenho.find("h2", text="Vasco da Gama Estatísticas do jogador")
    div_titulo = titulo_tabela.find_parent()
    tabela_desempenho = div_titulo.find_next_sibling().table.tbody
    # coletar dados da tabela
    dados_desempenho = [
        (
            # rodada
            numero_rodada,
            # jogador
            linha.th['data-append-csv'],
            # numero
            linha.find_all('td')[0].text,
            # idade
            linha.find_all('td')[3].text[:2],
            # minutos
            linha.find_all('td')[4].text,
            # gol
            linha.find_all('td')[5].text,
            # gol contra
            linha.find_all('td')[19].text,
            # gol de penalti
            linha.find_all('td')[7].text,
            # penalti perdido
            int(linha.find_all('td')[8].text) - int(linha.find_all('td')[7].text),
            # assistencia
            linha.find_all('td')[6].text,
            # total de chutes
            linha.find_all('td')[9].text,
            # chutes a gol
            linha.find_all('td')[10].text,
            # cartão amrelo
            linha.find_all('td')[11].text,
            # cartão vermelho
            linha.find_all('td')[12].text,
            # falta cometida
            linha.find_all('td')[13].text,
            # falta provocada
            linha.find_all('td')[14].text,
            # impedimento
            linha.find_all('td')[15].text,
            # cruxamento
            linha.find_all('td')[16].text,
            # roubada de bola
            linha.find_all('td')[17].text,
            # desarme
            linha.find_all('td')[18].text,
            # cobrança de penalti
            linha.find_all('td')[21].text,
            # penalti convertido
            linha.find_all('td')[20].text
        ) 
        for linha in tabela_desempenho.find_all("tr")
    ]
    # criar data frame e exportar csv
    desempenho_df = pd.DataFrame(dados_desempenho,
                                 columns = [
                                     "rodada",
                                     "jogador",
                                     "numero",
                                     "idade",
                                     "minutos",
                                     "gol",
                                     "gol_contra",
                                     "gol_penalti",
                                     "penalti_perdido",
                                     "assistencia",
                                     "chute_total",
                                     "chute_gol",
                                     "cartao_amarelo",
                                     "cartao_vermelho",
                                     "falta_cometida",
                                     "falta_provocada",
                                     "impedimento",
                                     "cruzamento",
                                     "roubada_bola",
                                     "desarme",
                                     "cobranca_penalti",
                                     "penalti_convertido"
                                 ])
    nome_csv = f'rodada{numero_rodada}_desempenho_jogadores.csv'
    desempenho_df.to_csv(nome_csv, index=False)
    print(f'Exportar CSV {nome_csv}')

Exportar CSV rodada 1_desempenho_jogadores.csv
Exportar CSV rodada 2_desempenho_jogadores.csv
Exportar CSV rodada 3_desempenho_jogadores.csv
Exportar CSV rodada 4_desempenho_jogadores.csv
Exportar CSV rodada 5_desempenho_jogadores.csv
Exportar CSV rodada 6_desempenho_jogadores.csv
Exportar CSV rodada 7_desempenho_jogadores.csv
Exportar CSV rodada 8_desempenho_jogadores.csv
Exportar CSV rodada 9_desempenho_jogadores.csv
Exportar CSV rodada10_desempenho_jogadores.csv
Exportar CSV rodada11_desempenho_jogadores.csv
Exportar CSV rodada12_desempenho_jogadores.csv
Exportar CSV rodada13_desempenho_jogadores.csv
Exportar CSV rodada14_desempenho_jogadores.csv
Exportar CSV rodada15_desempenho_jogadores.csv
Exportar CSV rodada16_desempenho_jogadores.csv
Exportar CSV rodada17_desempenho_jogadores.csv
Exportar CSV rodada18_desempenho_jogadores.csv
Exportar CSV rodada19_desempenho_jogadores.csv
Exportar CSV rodada20_desempenho_jogadores.csv
Exportar CSV rodada21_desempenho_jogadores.csv
Exportar CSV 

### Acessar páginas de desempenho e obter dados de goleiro

In [ ]:
dados_goleiro = []

for rodada in url_partidas:
    numero_rodada = rodada[0]
    url = rodada[1]
    # obter página
    pagina_desempenho = urlopen(f'https://fbref.com{url}')
    # simular tempo de execução de usuario comum
    sleep(60)
    bs_pagina_desempenho = BeautifulSoup(pagina_desempenho, 'html.parser')
    # localizar tabela de desempenho do goleiro do Vasco
    titulo_tabela_goleiro = bs_pagina_desempenho.find("h2", text="Vasco da Gama Estatísticas de goleiro")
    div_titulo_goleiro = titulo_tabela_goleiro.find_parent()
    tabela_desempenho_goleiro = div_titulo_goleiro.find_next_sibling().table.tbody
    # coletar dados da tabela
    dados_goleiro += [
        (
            # rodada
            numero_rodada,
            # jogador
            linha.th['csk'],
            # idade
            linha.find_all('td')[1].text[:2],
            # minutos
            linha.find_all('td')[2].text,
            # gol sofrido
            linha.find_all('td')[4].text,
            # defesa
            linha.find_all('td')[5].text,
        )
        for linha in tabela_desempenho_goleiro.find_all("tr")
    ]

# criar data frame e exportar csv
desempenho_goleiro_df = pd.DataFrame(dados_goleiro,
                                     columns = [
                                         "rodada",
                                         "jogador",
                                         "idade",
                                         "minutos",
                                         "gol_sofrido",
                                         "defesa"
                                     ])
desempenho_goleiro_df.to_csv("desempenho_goleiro.csv", index=False)